### Activité: Classifiez du texte

#### Chargement des articles de presse Reuters (à faire 1 seule fois avant de le sauvegarder avec le module pickle)

In [1]:
from sklearn.datasets import fetch_rcv1

rcv1 = fetch_rcv1()

In [2]:
#rcv1.data.shape
#rcv1.target.shape
#rcv1.sample_id
rcv1.target_names

#### Sauvegarde sur disque du jeu de données RCV1 (à faire 1 seule fois avant de le charger avec le module pickle)

In [3]:
import pickle
import os

path_to_save = 'C:\Formation\Data scientist\Projet_6\\TP\\reuters'
filename = path_to_save + os.sep + 'rcv1_data.sav'
pickle.dump(rcv1.data, open(filename, 'wb'))

In [4]:
filename = path_to_save + os.sep + 'rcv1_target.sav'
pickle.dump(rcv1.target, open(filename, 'wb'))
filename = path_to_save + os.sep + 'rcv1_target_names.sav'
pickle.dump(rcv1.target_names, open(filename, 'wb'))

#### On charge le dataset RCV1 avec le module pickle, car plus rapide que fetch_rcv1() de sklearn

In [46]:
import pickle
import os

path_to_load = 'C:\Formation\Data scientist\Projet_6\\TP\\reuters'
#filename = path_to_save + os.sep + 'rcv1_data.sav'
rcv1_data = pickle.load(open(os.path.join(path_to_load, 'rcv1_data.sav'), 'rb'))
#filename = path_to_save + os.sep + 'rcv1_target.sav'
rcv1_target = pickle.load(open(os.path.join(path_to_load, 'rcv1_target.sav'), 'rb'))
#filename = path_to_save + os.sep + 'rcv1_target_names.sav'
rcv1_target_names = pickle.load(open(os.path.join(path_to_load, 'rcv1_target_names.sav'), 'rb'))

#### 0n considère un échantillon de 5000 exemples pour des raisons de capacité calcul et mémoire

In [47]:
X = rcv1_data[:5000].toarray()
y = rcv1_target[:5000].toarray()
X.shape, y.shape

((5000, 47236), (5000, 103))

#### Suppression du jeu de données global RCV1 pour libérer de la mémoire

In [48]:
del rcv1_data, rcv1_target

### Création du jeu de données d'entrainement (et de test)

#### Préparation du jeu d'entrainement et de test

In [49]:
from sklearn import model_selection

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25, random_state=0)

In [50]:
X_train.shape, y_train.shape

((3750, 47236), (3750, 103))

### Nettoyage et préparation des données sur y_train et X_train

In [51]:
import pandas as pd
# Conversion de y_train en dataframe 
y_train_df = pd.DataFrame(y_train, columns=rcv1_target_names)

In [52]:
# Nbre d'articles par catégorie
nb_articles_per_categ = y_train_df.sum(axis=0)
nb_articles_per_categ.sort_values(ascending=False).head(10)

CCAT    1866
GCAT     983
MCAT     943
C15      774
ECAT     608
C151     452
M14      403
C152     341
E21      274
M13      262
dtype: int64

In [53]:
# Nbre de catégories par article
nb_categs_per_article = y_train_df.sum(axis=1)
nb_categs_per_article.sort_values(ascending=False).head(10)

2509    12
116     12
2042    11
2808    11
3665    11
3659    11
180     10
3669    10
845     10
2643    10
dtype: int64

#### On a pour ce jeu d'entrainement un nb. de catégories compris entre 1 et 12. 

#### On supprime les catégories qui apparaissent dans moins de 2% de la taille du dataset pour faciliter l'apprentissage des modèles

In [54]:
nb_categ_freq = 0.02 
categ_to_remove = nb_articles_per_categ[nb_articles_per_categ < nb_categ_freq*y_train_df.shape[0]]

In [55]:
# Nb. de catégories à supprimer
len(categ_to_remove)

67

#### Suppression des 67 catégories sur un total de 103

In [56]:
y_train_df.drop(list(categ_to_remove.index), axis=1, inplace=True)

#### Suppression des entrées de y_train et de X_train qui se retrouvent orphelines de catégories

In [57]:
# on supprime les entrées sans catégories
ix_zero_categ = list(y_train_df[(y_train_df == 0).all(axis=1)].index) 
y_train_df.drop(ix_zero_categ, axis=0, inplace=True)

In [58]:
ix_zero_categ

[]

In [59]:
# Conversion de X_train en dataframe 
X_train_df = pd.DataFrame(X_train)

In [60]:
# idem sur X_train, pour synchroniser avec y_train
X_train_df.drop(ix_zero_categ, axis=0, inplace=True)

#### Dimensions de X_train et y_train avant et après nettoyage des données

In [61]:
print('Avant cleaning: %s, %s' % (X_train.shape, y_train.shape))
print('Après cleaning: %s, %s' % (X_train_df.shape, y_train_df.shape))

Avant cleaning: (3750, 47236), (3750, 103)
Après cleaning: (3750, 47236), (3750, 36)


### Nettoyage et préparation des données sur y_test et X_test

In [62]:
# On reprend les mêmes opérations de data cleaning que pour y_train, X_train
y_test_df = pd.DataFrame(y_test, columns=rcv1_target_names)
y_test_df.drop(list(categ_to_remove.index), axis=1, inplace=True)

In [63]:
# on supprime les entrées sans catégories
ix_zero_categ = list(y_test_df[(y_test_df == 0).all(axis=1)].index) 
y_test_df.drop(ix_zero_categ, axis=0, inplace=True)

In [64]:
# Conversion de X_test en dataframe 
X_test_df = pd.DataFrame(X_test)
# idem sur X_test, pour synchroniser avec y_test
X_test_df.drop(ix_zero_categ, axis=0, inplace=True)

#### Dimensions de X_test et y_test avant et après nettoyage des données

In [65]:
print('Avant cleaning: %s, %s' % (X_test.shape, y_test.shape))
print('Après cleaning: %s, %s' % (X_test_df.shape, y_test_df.shape))

Avant cleaning: (1250, 47236), (1250, 103)
Après cleaning: (1250, 47236), (1250, 36)


#### Calcul de la densité de zéros dans le jeu de données X_train_df et y_train_df

In [66]:
import numpy as np

sparsity = 1 - np.sum(np.sum(y_train_df)) / (y_train_df.shape[0] * y_train_df.shape[1])
print('Pcentage de valeurs à 0 dans y_train_df : %.5f' % sparsity)
sparsity = 1 - np.sum(np.sum(X_train_df)) / (X_train_df.shape[0] * X_train_df.shape[1])
print('Pcentage de valeurs à 0 dans X_train_df : %.5f' % sparsity)

Pcentage de valeurs à 0 dans y_train_df : 0.92064
Pcentage de valeurs à 0 dans X_train_df : 0.99986


#### Le jeu de données d'entrainement reste extrêmement "creux" avec une densité de 0 très importante (92 % pour X_train_df et 99.9 % pour y_train)

---

### Modélisation

#### On est dans tous les cas face à un problème de classification binaire multi-output:
-> chaque article Reuters est catégorisé manuellement par 1 ou plusieurs tags (0 = absence du tag, 1 = présence du tag)

### 1. Test d'un classifieur Naive Bayes en mode multi-output

In [26]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB

nbayes = MultinomialNB()

moutput_nbayes = MultiOutputClassifier(nbayes, n_jobs=-1)
moutput_nbayes.fit(X_train_df, y_train_df)   

MultiOutputClassifier(estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
           n_jobs=-1)

In [29]:
y_pred = moutput_nbayes.predict(X_test_df)

#### On utilise 2 métriques pour mesurer l'efficacité du modèle sur la prédiction:
1) Accuracy score: donne le %tage d'articles pour lesquels l'ensemble des tags associés à chaque article a été correctement prédit <br>
2) Hamming loss: donne le %tage de tags incorrectement prédits sur l'ensemble des posts => est plus précis que l'accuracy score

In [31]:
from sklearn.metrics import hamming_loss, accuracy_score

print('Accuracy score: % .2f' % accuracy_score(y_pred, y_test_df) )
print('Hamming loss score: % .2f' % (1 - hamming_loss(y_pred, y_test_df)))

Accuracy score:  0.05
Hamming loss score:  0.94


#### Validation croisée sur l'accuracy score 

In [33]:
from sklearn.model_selection import cross_val_score
 
scores = cross_val_score(moutput_nbayes, X_train_df, y_train_df, cv=5)
print('Moyenne des erreurs du modèle :', scores.mean())
print('Variance des erreurs du modèle :', scores.std())

Moyenne des erreurs du modèle : 0.0434666666667
Variance des erreurs du modèle : 0.00645531649969


### 2. Test d'une Régression logistique en mode multi-output

In [35]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()

moutput_logreg = MultiOutputClassifier(logreg, n_jobs=-1)
moutput_logreg.fit(X_train_df, y_train_df)    

MultiOutputClassifier(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
           n_jobs=-1)

In [36]:
y_pred = moutput_logreg.predict(X_test_df)

#### Scores

In [37]:
from sklearn.metrics import hamming_loss, accuracy_score

print('Accuracy score: % .2f' % accuracy_score(y_pred, y_test_df) )
print('Hamming loss score: % .2f' % (1 - hamming_loss(y_pred, y_test_df)))

Accuracy score:  0.26
Hamming loss score:  0.96


#### Validation croisée sur l'accuracy score 

In [43]:
from sklearn.model_selection import cross_val_score
 
scores = cross_val_score(moutput_logreg, X_train_df, y_train_df, cv=5)
print('Moyenne des erreurs du modèle :', scores.mean())
print('Variance des erreurs du modèle :', scores.std())

Moyenne des erreurs du modèle : 0.2288
Variance des erreurs du modèle : 0.0116420693082


### 3. Test d'un SGD Classifier en mode multi-output

In [38]:
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier(loss="log", penalty='l1')

moutput_sgd = MultiOutputClassifier(sgd, n_jobs=-1)
moutput_sgd.fit(X_train_df, y_train_df)    

MultiOutputClassifier(estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l1', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False),
           n_jobs=-1)

In [40]:
y_pred = moutput_sgd.predict(X_test_df)

#### Scores

In [41]:
from sklearn.metrics import hamming_loss, accuracy_score

print('Accuracy score: % .2f' % accuracy_score(y_pred, y_test_df) )
print('Hamming loss score: % .2f' % (1 - hamming_loss(y_pred, y_test_df)))

Accuracy score:  0.45
Hamming loss score:  0.97


#### Validation croisée sur l'accuracy score 

In [44]:
from sklearn.model_selection import cross_val_score
 
scores = cross_val_score(moutput_sgd, X_train_df, y_train_df, cv=5)
print('Moyenne des erreurs du modèle :', scores.mean())
print('Variance des erreurs du modèle :', scores.std())

Moyenne des erreurs du modèle : 0.433866666667
Variance des erreurs du modèle : 0.0174986983643


---

#### Conclusion: il est possible d'obtenir un score satisfaisant du point de vue du Hamming loss avec les 3 classifieurs, sinon le SGD Classifier obtient le meilleur accuracy score (45 %) loin devant les deux autres et cela après entrainement sur seulement 3750 articles Reuters.
Dans tous les cas, l'accuracy score est bien plus contraignant car il faut pouvoir prédire pour un article Reuters TOUTES les catégories (soit les 36 catégories) de manière correcte sans exception. De plus il faudrait entrainer les modèles sur l'intégralité du dataset (800000 articles) pour avoir de meilleurs résultats mais les capacités machine ne me le permettent pas..

#### Il serait également intéressant de pouvoir réduire la dimension de X_train avec 47236, de plus cela permettrait de gagner en performance machine (calcul, mémoire..)